<a href="https://colab.research.google.com/github/fahriyegrl/ISIC-2024-skin-cancer-detection/blob/main/ISIC_2024_skin_cancer_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import torchvision.transforms as transforms
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
import pandas as pd
import h5py


train_metadata = pd.read_csv('https://github.com/fahriyegrl/ISIC-2024-skin-cancer-detection/raw/master/train-metadata.csv')
test_metadata = pd.read_csv('https://github.com/fahriyegrl/ISIC-2024-skin-cancer-detection/raw/main/test-metadata.csv')


train_metadata.head(), test_metadata.head()


<ipython-input-1-51f9f5f92d13>:8: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv('https://github.com/fahriyegrl/ISIC-2024-skin-cancer-detection/raw/master/train-metadata.csv')


(        isic_id  target  patient_id  age_approx   sex anatom_site_general  \
 0  ISIC_0015670       0  IP_1235828        60.0  male     lower extremity   
 1  ISIC_0015845       0  IP_8170065        60.0  male           head/neck   
 2  ISIC_0015864       0  IP_6724798        60.0  male     posterior torso   
 3  ISIC_0015902       0  IP_4111386        65.0  male      anterior torso   
 4  ISIC_0024200       0  IP_8313778        55.0  male      anterior torso   
 
    clin_size_long_diam_mm          image_type tbp_tile_type   tbp_lv_A  ...  \
 0                    3.04  TBP tile: close-up     3D: white  20.244422  ...   
 1                    1.10  TBP tile: close-up     3D: white  31.712570  ...   
 2                    3.40  TBP tile: close-up        3D: XP  22.575830  ...   
 3                    3.22  TBP tile: close-up        3D: XP  14.242329  ...   
 4                    2.73  TBP tile: close-up     3D: white  24.725520  ...   
 
     lesion_id  iddx_full  iddx_1  iddx_2  iddx_

In [12]:
import torch
import torchvision

torch.__version__, torchvision.__version__, torch.cuda.is_available()


('2.3.1+cu121', '0.18.1+cu121', False)

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [2]:
#Data transformation

skin_transform =  transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [3]:
!wget https://github.com/fahriyegrl/ISIC-2024-skin-cancer-detection/raw/master/train-metadata.csv

--2024-08-04 21:34:59--  https://github.com/fahriyegrl/ISIC-2024-skin-cancer-detection/raw/master/train-metadata.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/fahriyegrl/ISIC-2024-skin-cancer-detection/master/train-metadata.csv [following]
--2024-08-04 21:35:00--  https://media.githubusercontent.com/media/fahriyegrl/ISIC-2024-skin-cancer-detection/master/train-metadata.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 257543161 (246M) [text/plain]
Saving to: ‘train-metadata.csv’

train-metadata.csv  100%[===================>] 245.61M   164MB/s    in 1.5s    

2024-08-04 2

In [4]:
!wget https://github.com/fahriyegrl/ISIC-2024-skin-cancer-detection/raw/main/test-metadata.csv
!wget https://github.com/fahriyegrl/ISIC-2024-skin-cancer-detection/raw/master/train-image.hdf5
!wget https://github.com/fahriyegrl/ISIC-2024-skin-cancer-detection/raw/main/test-image.hdf5

--2024-08-04 21:35:06--  https://github.com/fahriyegrl/ISIC-2024-skin-cancer-detection/raw/main/test-metadata.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fahriyegrl/ISIC-2024-skin-cancer-detection/main/test-metadata.csv [following]
--2024-08-04 21:35:06--  https://raw.githubusercontent.com/fahriyegrl/ISIC-2024-skin-cancer-detection/main/test-metadata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2206 (2.2K) [text/plain]
Saving to: ‘test-metadata.csv’

test-metadata.csv   100%[===================>]   2.15K  --.-KB/s    in 0s      

2024-08-04 21:35:07 (13.0 MB/s) - ‘test-metadata.cs

In [5]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import os

# Paths to train/test HDF5 and CSV files
train_hdf5_file = 'train-image.hdf5'
test_hdf5_file = 'test-image.hdf5'
train_metadata_csv = 'train-metadata.csv'
test_metadata_csv = 'test-metadata.csv'

train_hdf5_file

'train-image.hdf5'

In [ ]:
#try:
#    with h5py.File(train_hdf5_file, 'r') as file:
#        print(f"Train HDF5 file successfully opened. Keys:\n{list(file.keys())}")
#except Exception as e:
#    print(f"Error opening {train_hdf5_file}: {e}")

In [6]:
train_metadata.dtypes


,0
isic_id,object
target,int64
patient_id,object
age_approx,float64
sex,object
anatom_site_general,object
clin_size_long_diam_mm,float64
image_type,object
tbp_tile_type,object
tbp_lv_A,float64


In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

#train_metadata.head(), test_metadata.head()
label_column = "target"
feature_columns = [col for col in train_metadata.columns if col != label_column and col != "isic_id" and col != "patient_id"
                   and col != "image_type" and col != "attribution" and col != "copyright_license"  ]

feature_columns_test = [col for col in test_metadata.columns if col != "isic_id" and col != "patient_id"
                   and col != "image_type" and col != "attribution" and col != "copyright_license"  ]


train_features = train_metadata[feature_columns]
train_labels = train_metadata[label_column]

test_features = test_metadata[feature_columns_test]
#test_labels = test_metadata[label_column]

categorical_features = train_features.select_dtypes(include=['object']).columns.tolist()
numerical_features = train_features.select_dtypes(include=['int64', 'float64']).columns.tolist()

categorical_features_test = test_features.select_dtypes(include=['object']).columns.tolist()
numerical_features_test = test_features.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor_train = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

preprocessor_test = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_test),
        ('cat', OneHotEncoder(), categorical_features_test)
    ])

preprocessor_train.fit(train_features)
train_features_processed = preprocessor_train.transform(train_features)

preprocessor_test.fit(test_features)
test_features_processed = preprocessor_test.transform(test_features)

train_features_processed_df = pd.DataFrame(train_features_processed)
test_features_processed_df = pd.DataFrame(test_features_processed)

train_features_processed_df.head()
#test_features_processed_df.head()






,0
0,"(0, 0)\t0.14614533641392255\n (0, 1)\t-0.51..."
1,"(0, 0)\t0.14614533641392255\n (0, 1)\t-1.62..."
2,"(0, 0)\t0.14614533641392255\n (0, 1)\t-0.30..."
3,"(0, 0)\t0.5138965705425961\n (0, 1)\t-0.407..."
4,"(0, 0)\t-0.22160589771475098\n (0, 1)\t-0.6..."


In [16]:
### Preparing the model by utilization pretrained VIT from torchvision.models
import torchvision
from torch import nn

vit_weights= torchvision.models.ViT_B_16_Weights.DEFAULT
vit_model = torchvision.models.vit_b_16(weights=vit_weights).to(device)

for param in vit_model.parameters():
    param.requires_grad = False

torch.manual_seed(42)

vit_model.heads = nn.Linear(in_features=768, out_features=2, bias=True).to(device)

In [17]:
## Getting transform from VIT weights
vit_transform = vit_weights.transforms()
print(vit_transform)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [8]:
import h5py
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import vit_b_16
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import io
from sklearn.preprocessing import StandardScaler

class SkinHDF5DatasetTrain(Dataset):
  def __init__(self, hdf5_file, metadata_csv, preprocessor, label_column, transform=None):
    self.file = h5py.File(hdf5_file, 'r')
    #print(self.file.keys())
    self.transform = transform
    self.metadata = pd.read_csv(metadata_csv)
    self.label_column = label_column
    self.preprocessor = preprocessor


    self.metadata.sort_values('isic_id', inplace=True)
    self.image_keys = self.metadata['isic_id'].tolist()

    metadata_features = self.metadata.drop(columns=[self.label_column, 'isic_id', 'patient_id', 'image_type', 'attribution', 'copyright_license'])
    self.features_processed = self.preprocessor.transform(metadata_features)

    self.labels = self.metadata[label_column].astype(int).values



  def __len__(self):
    return len(self.image_keys)

  def __getitem__(self, id):
    image_key = self.image_keys[id]
    image_bytes = self.file[image_key][()]
    image= Image.open(io.BytesIO(image_bytes)).convert('RGB')

    if self.transform:
      image = self.transform(image)

    label = self.labels[id]
    metadata = self.features_processed[id]

    label = torch.tensor(label, dtype=torch.long)
    metadata = torch.tensor(metadata.toarray(), dtype=torch.float32)

    return image, label, metadata


class SkinHDF5DatasetTest(Dataset):
  def __init__(self, hdf5_file, metadata_csv, preprocessor,transform=None):
    self.file = h5py.File(hdf5_file, 'r')
    self.transform = transform
    self.metadata = pd.read_csv(metadata_csv)
    self.label_column = label_column
    self.preprocessor = preprocessor


    self.metadata.sort_values('isic_id', inplace=True)
    self.image_keys = self.metadata['isic_id'].tolist()

    metadata_features = self.metadata.drop(columns=['isic_id', 'patient_id', 'image_type', 'attribution', 'copyright_license'])
    self.features_processed = self.preprocessor.transform(metadata_features)


  def __len__(self):
    return len(self.image_keys)

  def __getitem__(self, id):
    image_key = self.image_keys[id]
    image_bytes = self.file[image_key][()]
    image= Image.open(io.BytesIO(image_bytes)).convert('RGB')

    if self.transform:
      image = self.transform(image)

    label = self.labels[id]
    metadata = self.features_processed[id]

    label = torch.tensor(label, dtype=torch.long)
    metadata = torch.tensor(metadata.toarray(), dtype=torch.float32)

    return image, label, metadata




In [18]:
# Initialize datasets and dataloaders
train_dataset = SkinHDF5DatasetTrain(train_hdf5_file,train_metadata_csv, preprocessor_train,label_column, transform=vit_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = SkinHDF5DatasetTest(test_hdf5_file, test_metadata_csv,preprocessor_test, transform=vit_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

<ipython-input-8-6371251e551a>:18: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  self.metadata = pd.read_csv(metadata_csv)


In [11]:
#for image, label, metadata in train_loader:
 #   print("Sample image tensor values:", image[0])  # Print the values of the first image tensor
 #   print("Sample label value:", label[0].item())  # Print the first label value
 #   print("Sample metadata values:", metadata[0])

In [ ]:
## Training the model
import torch
from tqdm.auto import tqdm

optimizer = torch.optim.Adam(vit_model.parameters(), lr=0.001)
loss_fn = nn.BCEWithLogitsLoss()
epoch = 10


train_loss, train_acc, test_loss, test_acc = 0, 0 ,0 ,0
results = {train_loss: [],
          train_acc: [],
          test_loss: [],
          test_acc: []}
for epoch in tqdm(range(epoch)):
  vit_model.train()
  for batch, (X,y) in enumerate(train_loader):
      X, y = X.to(device), y.to(device)
      y_pred = vit_model(X).squeeze()
      y_onehot = torch.nn.functional.one_hot(y, num_classes=2).float().to(device)
      loss= loss_fn(y_pred, y_onehot)
      train_loss += loss.item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      y_pred_probs = torch.sigmoid(y_pred)
      y_pred_class = torch.round(y_pred_probs).argmax(dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  train_loss = train_loss / len(train_loader)
  train_acc = train_acc / len(train_loader)
  #print(f"Train loss: {train_loss:.2f} Train accuracy: {train_acc:.2f}")

  vit_model.eval()
  with torch.inference_mode():
    for batch, (X,y) in enumerate(test_loader):
      X, y = X.to(device), y.to(device)
      test_pred_logits = vit_model(X).squeeze()
      t_loss = loss_fn(test_pred_logits, y)
      test_loss += t_loss.item()
      test_pred_probs = torch.sigmoid(test_pred_logits)
      test_pred_labels = torch.round(test_pred_probs).argmax(dim=1)
      test_acc += (test_pred_labels == y).sum().item()/len(test_pred_labels)

  test_loss = test_loss / len(test_loader)
  test_acc = test_acc / len(test_loader)
  print(f"Epoch: {epoch+1}/{epoch} | train_loss: {train_loss} | train_acc: {train_acc} |test_loss: {test_loss} | test_acc: {test_acc}")
 #print(f"Test loss: {test_loss:.2f} Test accuracy: {test_acc:.2f}")
  results[train_loss].append(train_loss)
  results[train_acc].append(train_acc)
  results[test_loss].append(test_loss)
  results[test_acc].append(test_acc)

In [ ]:
## Plotting results to see if there is any over/under fitting
import matplotlib.pyplot as plt

loss = results['train_loss']
test_loss = results['test_loss']

accuracy= results['train_acc']
test_accuracy = results['test_acc']

epoch = range(len(results['train_loss']))

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epoch, loss, label='Training Loss')
plt.plot(epoch, test_loss, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epoch, accuracy, label='Training Accuracy')
plt.plot(epoch, test_accuracy, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()


In [ ]:
from pathlib import Path

target_dir_path = Path("vit_model")
target_dir_path.mkdir(parents=True, exist_ok=True)

model_save_path = target_dir_path / "vit_model.pth"
torch.save(vit_model.state_dict(), model_save_path)